In [1]:
import numpy as np
import os

from dingodb import SDKVectorDingoDB, SDKClient
from dingodb.common.vector_rep import ScalarType, ScalarColumn, ScalarSchema

addrs = "127.0.0.1:22001,127.0.0.1:22002,127.0.0.1:22003"
sdk_client = SDKClient(addrs)
x = SDKVectorDingoDB(sdk_client)
print(x)

index_name = "test_index_grpc"

In [2]:
# delete_index
x.delete_index(index_name)

True

In [3]:

# create_index
# help(x.create_index)
# x.create_index(index_name, 6, "hnsw", "euclidean", 3, index_config={"efConstruction": 300,"maxElements": 60000,"nlinks": 64}, operand=None)
# x.create_index(index_name, 6, "hnsw", "euclidean", 3, index_config={"efConstruction": 300,"maxElements": 60000,"nlinks": 64}, operand=[])
x.create_index(index_name, 6, "hnsw", "euclidean", 3, index_config={"efConstruction": 300,"maxElements": 60000,"nlinks": 64}, operand=[5,10,15,20])


True

In [4]:
# make dataset
# 构建训练数据和查询数据
d = 6                           # dimension
nb = 4                      # database size
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
print(xb)
print(xb.shape)
xb[:, 0] += np.arange(nb) / 1000.
print(xb)
print(xb.shape)

ids = [1, 2, 3, 4]
datas = [{"a1": "b1"}, {"a2": "b2"}, {"a3": "b3"}, {"a4": "b4"}]
vectors = xb.tolist()

[[0.19151945 0.62210876 0.43772775 0.7853586  0.77997583 0.2725926 ]
 [0.27646425 0.8018722  0.95813936 0.87593263 0.35781726 0.5009951 ]
 [0.6834629  0.71270204 0.37025076 0.5611962  0.50308317 0.01376845]
 [0.7728266  0.8826412  0.364886   0.6153962  0.07538124 0.368824  ]]
(4, 6)
[[0.19151945 0.62210876 0.43772775 0.7853586  0.77997583 0.2725926 ]
 [0.27746424 0.8018722  0.95813936 0.87593263 0.35781726 0.5009951 ]
 [0.6854629  0.71270204 0.37025076 0.5611962  0.50308317 0.01376845]
 [0.77582663 0.8826412  0.364886   0.6153962  0.07538124 0.368824  ]]
(4, 6)


In [5]:
# vector_add
x.vector_add(index_name, datas, vectors, ids)

[{'id': 1,
  'vector': {'dimension': 6,
   'valueType': 'FLOAT',
   'floatValues': [0.19151945412158966,
    0.6221087574958801,
    0.43772774934768677,
    0.7853586077690125,
    0.7799758315086365,
    0.27259260416030884],
   'binaryValues': []},
  'scalarData': {'a1': {'fieldType': 'STRING', 'fields': ['b1']}}},
 {'id': 2,
  'vector': {'dimension': 6,
   'valueType': 'FLOAT',
   'floatValues': [0.27746424078941345,
    0.801872193813324,
    0.9581393599510193,
    0.8759326338768005,
    0.35781726241111755,
    0.5009950995445251],
   'binaryValues': []},
  'scalarData': {'a2': {'fieldType': 'STRING', 'fields': ['b2']}}},
 {'id': 3,
  'vector': {'dimension': 6,
   'valueType': 'FLOAT',
   'floatValues': [0.6854628920555115,
    0.7127020359039307,
    0.37025076150894165,
    0.5611962080001831,
    0.5030831694602966,
    0.013768449425697327],
   'binaryValues': []},
  'scalarData': {'a3': {'fieldType': 'STRING', 'fields': ['b3']}}},
 {'id': 4,
  'vector': {'dimension': 6,
  

In [6]:
# vector_search
x.vector_search(index_name, vectors[0])
# return 
    # error RuntimeError

[{'vectorWithDistances': [{'id': 1,
    'vector': {'dimension': 6,
     'valueType': 'FLOAT',
     'floatValues': [0.19151945412158966,
      0.6221087574958801,
      0.43772774934768677,
      0.7853586077690125,
      0.7799758315086365,
      0.27259260416030884],
     'binaryValues': []},
    'scalarData': {'a1': {'fieldType': 'STRING', 'fields': ['b1']}},
    'distance': 0.0,
    'metricType': 'METRIC_TYPE_L2'},
   {'id': 3,
    'vector': {'dimension': 6,
     'valueType': 'FLOAT',
     'floatValues': [0.6854628920555115,
      0.7127020359039307,
      0.37025076150894165,
      0.5611962080001831,
      0.5030831694602966,
      0.013768449425697327],
     'binaryValues': []},
    'scalarData': {'a3': {'fieldType': 'STRING', 'fields': ['b3']}},
    'distance': 0.4506487250328064,
    'metricType': 'METRIC_TYPE_L2'},
   {'id': 2,
    'vector': {'dimension': 6,
     'valueType': 'FLOAT',
     'floatValues': [0.27746424078941345,
      0.801872193813324,
      0.9581393599510193,


In [7]:
# vector_search with pre_filter or post_filter
x.vector_search(index_name, vectors[0], 10, {"meta_expr": {"a1": "b1"}})

[{'vectorWithDistances': [{'id': 1,
    'vector': {'dimension': 6,
     'valueType': 'FLOAT',
     'floatValues': [0.19151945412158966,
      0.6221087574958801,
      0.43772774934768677,
      0.7853586077690125,
      0.7799758315086365,
      0.27259260416030884],
     'binaryValues': []},
    'scalarData': {'a1': {'fieldType': 'STRING', 'fields': ['b1']}},
    'distance': 0.0,
    'metricType': 'METRIC_TYPE_L2'}]}]

In [8]:
# get index with id
x.vector_get(index_name, [1, 2, 6])

[{'id': 1,
  'vector': {'dimension': 6,
   'valueType': 'FLOAT',
   'floatValues': [0.19151945412158966,
    0.6221087574958801,
    0.43772774934768677,
    0.7853586077690125,
    0.7799758315086365,
    0.27259260416030884],
   'binaryValues': []},
  'scalarData': {'a1': {'fieldType': 'STRING', 'fields': ['b1']}}},
 {'id': 2,
  'vector': {'dimension': 6,
   'valueType': 'FLOAT',
   'floatValues': [0.27746424078941345,
    0.801872193813324,
    0.9581393599510193,
    0.8759326338768005,
    0.35781726241111755,
    0.5009950995445251],
   'binaryValues': []},
  'scalarData': {'a2': {'fieldType': 'STRING', 'fields': ['b2']}}},
 None]

In [9]:
# vector delete with id
x.vector_delete(index_name, [1, 2, 3])

[True, True, True]

In [10]:
x.vector_search(index_name, vectors[0])

[{'vectorWithDistances': [{'id': 4,
    'vector': {'dimension': 6,
     'valueType': 'FLOAT',
     'floatValues': [0.775826632976532,
      0.8826411962509155,
      0.36488598585128784,
      0.6153962016105652,
      0.07538124173879623,
      0.3688240051269531],
     'binaryValues': []},
    'scalarData': {'a4': {'fieldType': 'STRING', 'fields': ['b4']}},
    'distance': 0.9491992592811584,
    'metricType': 'METRIC_TYPE_L2'}]}]

In [20]:
# Add: scan
x.vector_scan(index_name, 1, 2)

RuntimeError: scan index test_index_grpc fail: NotFound (errno:30010) : index_name not found

In [12]:
# Add vector count 
x.vector_count(index_name)

1

In [13]:
x.vector_metrics(index_name)

{'indexType': 'Hnsw',
 'count': 4,
 'deletedCount': 3,
 'maxVectorId': 4,
 'minVectorId': 4,
 'memoryBytes': 2688}

In [14]:
x.get_max_index_row(index_name)

4

In [15]:
# delete_index
x.delete_index(index_name)

True

In [16]:
scheme =  ScalarSchema()
col = ScalarColumn("id", ScalarType.STRING, True)
scheme.add_scalar_column(col)
col = ScalarColumn("name", ScalarType.STRING, False)
scheme.add_scalar_column(col)

x.create_index_with_schema(index_name, 6, scheme, "hnsw", "euclidean", 3, index_config={"efConstruction": 300,"maxElements": 60000,"nlinks": 64}, operand=[5,10,15,20])



True

In [17]:

# make dataset
# 构建训练数据和查询数据
d = 6                           # dimension
nb = 4                      # database size
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
print(xb)
xb[:, 0] += np.arange(nb) / 1000.
print(xb)

ids = [1, 2, 3, 4]
datas = [{"id": "1", "name": "1"}, {"id": "2", "name": "2"}, {"id": "3", "name": "3"}, {"id": "4", "name": "4"}]
vectors = xb.tolist()

x.vector_add(index_name, datas, vectors, ids)

[[0.19151945 0.62210876 0.43772775 0.7853586  0.77997583 0.2725926 ]
 [0.27646425 0.8018722  0.95813936 0.87593263 0.35781726 0.5009951 ]
 [0.6834629  0.71270204 0.37025076 0.5611962  0.50308317 0.01376845]
 [0.7728266  0.8826412  0.364886   0.6153962  0.07538124 0.368824  ]]
[[0.19151945 0.62210876 0.43772775 0.7853586  0.77997583 0.2725926 ]
 [0.27746424 0.8018722  0.95813936 0.87593263 0.35781726 0.5009951 ]
 [0.6854629  0.71270204 0.37025076 0.5611962  0.50308317 0.01376845]
 [0.77582663 0.8826412  0.364886   0.6153962  0.07538124 0.368824  ]]


[{'id': 1,
  'vector': {'dimension': 6,
   'valueType': 'FLOAT',
   'floatValues': [0.19151945412158966,
    0.6221087574958801,
    0.43772774934768677,
    0.7853586077690125,
    0.7799758315086365,
    0.27259260416030884],
   'binaryValues': []},
  'scalarData': {'id': {'fieldType': 'STRING', 'fields': ['1']},
   'name': {'fieldType': 'STRING', 'fields': ['1']}}},
 {'id': 2,
  'vector': {'dimension': 6,
   'valueType': 'FLOAT',
   'floatValues': [0.27746424078941345,
    0.801872193813324,
    0.9581393599510193,
    0.8759326338768005,
    0.35781726241111755,
    0.5009950995445251],
   'binaryValues': []},
  'scalarData': {'id': {'fieldType': 'STRING', 'fields': ['2']},
   'name': {'fieldType': 'STRING', 'fields': ['2']}}},
 {'id': 3,
  'vector': {'dimension': 6,
   'valueType': 'FLOAT',
   'floatValues': [0.6854628920555115,
    0.7127020359039307,
    0.37025076150894165,
    0.5611962080001831,
    0.5030831694602966,
    0.013768449425697327],
   'binaryValues': []},
  'sca

In [18]:

# get index with id
# x.vector_get(index_name, [1, 2, 3, 4])
x.vector_get(index_name, [1, 2])

[{'id': 1,
  'vector': {'dimension': 6,
   'valueType': 'FLOAT',
   'floatValues': [0.19151945412158966,
    0.6221087574958801,
    0.43772774934768677,
    0.7853586077690125,
    0.7799758315086365,
    0.27259260416030884],
   'binaryValues': []},
  'scalarData': {'id': {'fieldType': 'STRING', 'fields': ['1']},
   'name': {'fieldType': 'STRING', 'fields': ['1']}}},
 {'id': 2,
  'vector': {'dimension': 6,
   'valueType': 'FLOAT',
   'floatValues': [0.27746424078941345,
    0.801872193813324,
    0.9581393599510193,
    0.8759326338768005,
    0.35781726241111755,
    0.5009950995445251],
   'binaryValues': []},
  'scalarData': {'id': {'fieldType': 'STRING', 'fields': ['2']},
   'name': {'fieldType': 'STRING', 'fields': ['2']}}}]

In [19]:

# delete_index
x.delete_index(index_name)

True